In [117]:
import requests
import urllib
import csv
from bs4 import BeautifulSoup
from datetime import datetime
import os

In [118]:
def get_topic_list_html(pagenum):
    r = requests.get('http://www.kdi.re.kr/policy/ep_list.jsp?pg='+str(pagenum)+'&&pp=10')
    
    soup = BeautifulSoup(r.text, "lxml")
    return soup

In [119]:
dataset = []

In [120]:
def get_data(html):
    title_list = []
    origin_list = []
    link_list = []
    
    tmp = html.find_all(class_="rpt_conts")[0]
    tmp_li = tmp.find_all('a')
    
    for i in range(0,len(tmp_li)):
        tmp_str = str(tmp_li[i]).split(">")[1]
        tmp_str = str(tmp_str).split("<")[0]
        title_list.append(tmp_str)
        
    tmp_li = tmp.find_all(class_="rpt_sup")
    
    for i in range(0, len(tmp_li)):
        tmp_str = str(tmp_li[i]).split("<em>")[1]
        tmp_ori = str(tmp_str).split("</em>")[0]
        origin_list.append(tmp_ori)
        
    tmp_li = tmp.find_all(class_="rpt_lst")
    
    for i in range(0, len(tmp_li)):
        tmp_str = str(tmp_li[i]).split(">")[1]
        tmp_str = str(tmp_str).split("\"")[3]
        tmp_str= str(tmp_str).split("./")[1]
        tmp_link = "www.kdi.re.kr/policy/" + tmp_str
        
        link_list.append(tmp_link)
    
    for i in range(0, len(tmp_li)):
        dataset.append({ "구분": "중앙부처", "발표처": origin_list[i], "제목": title_list[i], "웹주소": link_list[i]})
        


In [121]:
test_html = get_topic_list_html(1)
get_data(test_html)

In [127]:
date = str(datetime.today()).split(" ")[0]

In [128]:
def WriteDictToCSV(csv_file, csv_columns, dict_data):
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError as (errno, strerror):
            print("I/O error({0}): {1}".format(errno, strerror)) 
    return     


In [129]:
csv_columns = ["구분", "발표처", "제목", "웹주소"]
currentPath = os.getcwd()
csv_file = date + ".csv"
WriteDictToCSV(csv_file, csv_columns, dataset)
